# Business Inteligence Sheet 7

### Juan David Camacho
### Hernán Russi
### Juan Esteban Daza  

## Exercise 1 (3 Points)
1. Show that, if $S$, $T$ are tidsets, then $i(ST) = i(S) ∩ i(T)$, and even $i(S) ⊇ i(T)$ if $S ⊆ T$
2. Let $X$, $Y$ be two disjoint itemsets, $Z ⊇ XY$ with $sup(Z) = sup(XY )$, and $W ⊂ X$.
Show that conf $(W \rightarrow (Z \ W )) ≤ conf (X \rightarrow Y )$ and explain why this is important
for the AssociationRules algorithm.
3. Consider two rules $X \rightarrow Y$ and $W \rightarrow Y$ such that $W ⊂ X$. One might expect that
the confidence is monotonic in the premise, i.e., that the fact that $X ⊃ W$ implies that
$conf(X \rightarrow Y ) ≥ conf(W \rightarrow Y )$. That is, adding knowledge to the premise should
not reduce the confidence of the rule.
Give an example that shows that this is not necessarily the case.

## Exercise 2 (5 Points)
In this exercise, we will use the transaction view on itemsets. Note that we treat itemsets as python lists (not numpy arrays). Hence, in the following db refers to a list of lists (a list of transactions, which are encoded as lists).

1. Write functions apriori(db, minsup) and eclat(db, minsup) that run the respective
algorithms and return the set of frequent itemsets (as tuples of sets with their support,
using the threshold minsup). Run both algorithms on the shop dataset and report the
runtimes.

In [8]:
from template import read_database
from template import diff
from template import powerset
import numpy as np

x = read_database('example.dat')
#x = read_database('example2.dat')
y = diff(x[0], x[1])
z = powerset(x[0])

def get_prefix_tree_initial_single_items(db):
    c_0 = []
    for listiiiiiiiiiiii in db:
        for sublist in listiiiiiiiiiiii:
            c_0.append(sublist)

    unique_items = sorted(set(c_0))
    returned_list = []
    for value in unique_items:
        auxilar_list = []
        auxilar_list.append(value)
        returned_list.append(auxilar_list)
    return returned_list
    

def extend_prefix_tree(tree_level, k):

    c_0 = []
    for listiiiiiiiiiiii in tree_level:
        for sublist in listiiiiiiiiiiii:
            c_0.append(sublist)

    next_level_macro = powerset(c_0)
    next_level = []
    for sublist in next_level_macro:
        if len(sublist) == k:
            next_level.append(sublist)

    return sorted(next_level)

def compute_support(db, evaluated_itemset):
    sup = 0
    for row in db:
        if all(value in row for value in evaluated_itemset):
           sup += 1 
    return sup

def apriori(db, minsup):

    F = [] #Frequent k-itemsets
    C = [] #Prefix Tree
    C.append(get_prefix_tree_initial_single_items(db))

    k = 0
    length = 2
    while len(C[k]) != 0:
        supports = []
        for evaluated_itemset in C[k]:
            supports.append(compute_support(db, evaluated_itemset))
        
        #lines 7 - 9
        auxiliar  = C[k][:]
        auxiliarS = supports[:]
        for leaf in C[k]:
            if supports[C[k].index(leaf)] >= minsup:
                F.append((leaf, supports[C[k].index(leaf)]))
            else:
                auxiliarS.pop(auxiliar.index(leaf))
                auxiliar.remove(leaf)
        C[k] = auxiliar
        supports = auxiliarS
        #lines 7 - 9
        C.append(extend_prefix_tree(C[0], length))
        k += 1
        length += 1
    return F

In [2]:
def eclat(db, minsup):

    F = []
    P = []

    c_0 = []
    for listiiiiiiiiiiiiiii in db:
        for sublist in listiiiiiiiiiiiiiii:
            c_0.append(sublist)

    power_set_C0 = powerset(sorted(set(c_0)))
    tree_first_level = sorted([value for value in power_set_C0 if len(value) == 1])

    t_i_macro = []
    for i in range(len(tree_first_level)):
        t_i = []
        for row in db:
            if all(value in row for value in tree_first_level[i]):
                t_i.append(db.index(row))
        t_i_macro.append(sorted(t_i))
    
    for i in range(len(tree_first_level)):
        if len(t_i_macro[i]) >= minsup:
            P.append((tree_first_level[i], t_i_macro[i]))

    def eclat2(P, minsup, F):
        #for k in range(5):
        for p in P:
            F.append((p[0], len(p[1])))
            P_a = []
            for p2 in P:
                if P.index(p2) > P.index(p):
                    Xab = sorted(list(set(p[0])) + list(set(p2[0])))
                    t_Xab = [value for value in p[1] if value in p2[1]]
                    if len(t_Xab) >= minsup:
                        P_a.append((sorted(set(Xab)),t_Xab))
            if len(P_a) != 0:
                eclat2(P_a, minsup, F)
            #P.append(P_a)
    
    eclat2(P, minsup, F)
                
    return(F)



In [6]:
import timeit
shop_dataset_macro = read_database('shop.dat')
print('terminó de leer')
shop_dataset = shop_dataset_macro[0:1000]
start = timeit.default_timer()
apriori(shop_dataset, 500)
stop = timeit.default_timer()
apriori_timer = stop - start

start = timeit.default_timer()
eclat(shop_dataset, 500)
stop = timeit.default_timer()
eclat_timer = stop - start


print('Runtime Aglo Clustering single: ', apriori_timer)
print('Runtime Aglo Clustering LW single: ', eclat_timer)

terminó de leer


MemoryError: 

2. Write functions createAssociationRules(fsets, minconf) and getStrongRules(db,
minsup, minconf). The first should implement the AssociationRules algorithm and
compute all association rules with a given confidence from a set of frequent sets in the
form (premise, conclusion, abs sup, conf). The second should compute all frequent sets and then derives the strong rules from it with respect to the given minimum
confidence.

In [37]:
def createAssociationRules(fsets, minconf):

    AMPB = []

    for frequent in fsets:
        fsets_items = [x[0] for x in fsets]
        A = []
        provisional_supp = []
        if len(frequent[0]) >= 2:
            powerset_frequent = powerset(frequent[0])
            for powerset_item in powerset_frequent:
                if powerset_item in fsets_items and powerset_frequent.index(powerset_item) > 0:
                    A.append(powerset_item)
                    provisional_supp.append(fsets[fsets_items.index(powerset_item)][1])

            while len(A) != 0:
                X = A[-1]
                Y = diff(frequent[0], X)
                supX = provisional_supp[A.index(X)]
                A.remove(X)
                c = frequent[1]/supX

                if c >= minconf:
                    AMPB.append((X, Y, frequent[1], c))
                else:
                    powerset_remove = powerset(X)
                    for powerset_remove_item in powerset_remove:
                        if powerset_remove_item in A:
                            A.remove(powerset_item)
    
    return AMPB
                    

def getStrongRules(db, minsup, minconf):

    F = eclat(db, minsup)

    JS = createAssociationRules(F, minconf)

    return JS

3. Your boss is interested in a list of strong rules of the shop dataset. He considers rules
strong if their support is at least 500 and their confidence is at least .75. Report a list
of such strong rules sorted by support. He is also interested in a sub-list of these strong
rules in which at least two items appear in the conclusion of the rule. Report also this
sub-list, ordered descendingly by the number of items in the conclusion.
State explicitly, e.g. in a markdown cell, what the first of these rules means, and explain
what the respective support and confidence value of that specific rule mean.
Hint: Use a Pandas DataFrame to store the rules and to operate on them